In [ ]:
from visualize_data import json_to_dataframe
import re
import pandas as pd
import numpy as np

In [ ]:
df = json_to_dataframe("js")

In [ ]:
# 레벨별

def multi_index_to_dict(data):
    temp_dict = dict()
    for (index_1,index_2), value in data.items():
        if index_1 not in temp_dict:
            temp_dict[index_1] = {}
        temp_dict[index_1][index_2]  = value
    return temp_dict


def get_level_dict(df):
    
    # 레벨별 문제 수 : 지금까지 푼 문제 중 레벨별 문제 id의 개수를 의미합니다.
    temp_df = df.copy()
    temp_df["problem_name"] = temp_df["problem_name"].apply(lambda x: re.sub("\(1\)|\s", "", x))
    temp_df = temp_df.groupby("level")["problem_name"].nunique()
    level_problem_name = temp_df.to_dict()

    # 레벨별 고유한 문제 유형 개수 : 총 13개의 문제 유형 중 레벨별 문제 유형 id의 개수를 의미합니다.
    temp_df = df.copy()
    temp_df["problem_type"] = temp_df["problem_type"].apply(lambda x: re.sub("\(1\)|\s", "", x))
    temp_df = temp_df.groupby("level")["problem_type"].nunique()
    level_problem_type = temp_df.to_dict()
    
    # 레벨별 문제 유형 비율 : 사람들이 레벨별 어떤 문제 유형을 많이 풀었는지 살펴보실 수 있습니다.
    temp_df = df.copy()
    temp_df["file_name"] = temp_df["file_name"].apply(lambda x: re.sub("\s", "", x))
    temp_df["problem_type"] = temp_df["problem_type"].apply(lambda x: re.sub("\(1\)|\s", "", x))
    temp_df = temp_df.groupby(["level","problem_type"])["file_name"].nunique()
    level_problem_type_ratio = multi_index_to_dict(temp_df)
    
    # 레벨별 메소드 이용수, 누적 : 레벨별로 어떤 메소드를 주로 사용하여 문제를 풀이했는지 살펴보실 수 있습니다.
    temp_df = df.copy()
    temp_df = df.groupby(['level','function_method'])['countmethod'].sum()
    level_per_function_method = multi_index_to_dict(temp_df)

    # 메소드 이용 현황, 누적 값 : 문제 풀이 시, 어떤 메소드를 많이 사용했는지 순위 표를 보실 수 있습니다.
    temp_df = df.copy()
    function_method_sort = temp_df.groupby("function_method")["countmethod"].sum().sort_values(ascending = False).nlargest(10).to_dict()
    
    return level_problem_name, level_problem_type, level_problem_type_ratio, level_per_function_method, function_method_sort

In [ ]:
# 연도별

def NaN_to_None(d):
    return {k1 : {k2: None if v2 == -1 else v2 for k2, v2 in v1.items()} \
                    for k1, v1 in d.items()}


def get_year_dict(df):

    # 연도 별 수집 문제 수
    temp_df = df.copy()
    temp_df = temp_df.loc[temp_df.year!='0000']
    temp_df = temp_df.drop_duplicates('problem_name')
    temp_df = temp_df.groupby('year')['file_name'].count()
    years = temp_df.to_dict()


    # 연도 별 문제 유형 수
    temp_df = df.copy()
    temp_df = temp_df.loc[temp_df.year!='0000']
    temp_df = temp_df.drop_duplicates('problem_name')
    temp_df = temp_df.sort_values('year', ascending=False)

    year_type = temp_df.groupby(['year', 'problem_type']).size().unstack().fillna(-1).to_dict()
    year_type = NaN_to_None(year_type)

    
    return years, year_type

In [ ]:
# 출제업체별

def NaN_to_None(d):
    return {k1 : {k2: None if v2 == -1 else v2 for k2, v2 in v1.items()} \
                    for k1, v1 in d.items()}

def get_c_dict(df):
    
    # 회사별 문제수
    temp_df = df.copy()
    temp_df = temp_df.drop_duplicates('problem_name')
    
    companys = temp_df.groupby('company_name').count().iloc[:,:1].to_dict()['file_name']
    

    # 회사별 문제 유형
    temp_df = df.copy()
    temp_df = temp_df.drop_duplicates('problem_name')
    
    company_type = temp_df.groupby(['problem_type','company_name'])['file_name'].count().unstack().fillna(-1).to_dict()
    company_type = NaN_to_None(company_type)
    
    # 회사 별 문제 유형 비율 (라이다)

    temp_df = df.copy()
    temp_df = temp_df.drop_duplicates('problem_name')
    temp_df = temp_df.loc[temp_df.problem_type != '구현']
    temp_df = temp_df.groupby(['problem_type','company_name'])['file_name'].count().unstack()
    company_ptype_radar = (temp_df/temp_df.sum()).fillna(0).to_dict()
    
    return companys, company_type, company_ptype_radar

In [ ]:
# 문제유형별

def df_to_dict(df):
    '''
    분석 시각화 데이터를 딕셔너리 자료형으로 변환
    '''
    
    if df.index.nlevels == 1:
        return df.to_dict()

    elif df.index.nlevels == 2:
        temp_dict = {}
        for (first_key, second_key), value in df.items():
            if first_key not in temp_dict:
                temp_dict[first_key] = {}
            temp_dict[first_key][second_key] = value
        return temp_dict


def get_ptype_dict(df):
    '''
    JavaScript 문제의 문제유형별 문제 수를 카운트하는 함수입니다.
    JavaScript 문제의 문제유형마다 사용된 메소드별 횟수를 카운트하는 함수입니다.
    '''

    # 문제의 문제유형별 문제 수

    temp_df = df.copy()
    temp_df = temp_df.drop_duplicates(['file_name'], keep='first')
    temp_df = temp_df['problem_type'].value_counts()

    problem_type_count = df_to_dict(temp_df)


    # 문제유형별 사용된 메소드별 횟수
    temp_df = df.copy()
    temp_df = temp_df[temp_df["function_method"] != 0]
    temp_df = temp_df[temp_df["problem_type"] != "구현"] # 현 데이터에 문제유형 중 '구현'(문제유형 불분명)이 상당 부분 차지하므로 임시 제외
    temp_df = temp_df.groupby(['problem_type', 'function_method'])['countmethod'].sum()
    
    problem_type_function_method = df_to_dict(temp_df)
    
    return problem_type_count, problem_type_function_method

In [ ]:
# 전체 데이터 저장하는 모듈

def get_dict(df):
    '''
    JavaScript 문제의 전체 데이터를 dictionary 형태로 변환하여 반환하는 함수입니다.
    '''    

    # 전체 데이터 저장할 딕셔너리
    temp_dict = {}
    
    # 딕셔너리에 파트별 데이터 삽입
    
    level_problem_name, level_problem_type, level_problem_type_ratio, level_per_function_method, function_method_sort = get_level_dict(df)  # 레벨별 & 전체적인 메서드 사용 현황
    problem_type_count, problem_type_function_method = get_ptype_dict(df)  # 문제유형별

    years, year_type = get_year_dict(df) # 연도별
    companys, company_type, company_ptype_radar = get_c_dict(df) # 출제업체별
    
    # level 별
    temp_dict['level_problem_name'] = level_problem_name
    temp_dict['level_problem_type'] = level_problem_type
    temp_dict['level_problem_type_ratio'] = level_problem_type_ratio
    temp_dict['level_per_function_method'] = level_per_function_method
    temp_dict['function_method_sort'] = function_method_sort

    # 문제유형별
    temp_dict['problem_type_count'] = problem_type_count
    temp_dict['problem_type_function_method'] = problem_type_function_method
    
    # 연도별
    temp_dict['years'] = years
    temp_dict['year_type'] = year_type

    # 출제업체별
    temp_dict['companys'] = companys
    temp_dict['company_type'] = company_type
    temp_dict['company_ptype_radar'] = company_ptype_radar
    
    return temp_dict

In [ ]:
# main.py 에서 실행되는 코드
# 작업하실때 해당 셀 코드 활용하시면 됩니다!

# Standard library import
import os
import json

# Local application library
from visualize_data import json_to_dataframe
from visualize_data import py_get_dict
from visualize_data import js_get_dict
from visualize_data.get_description import description_to_dict


def get_viz_json():
    # load json and transform into dataframe
    py_df = json_to_dataframe("py")
    js_df = json_to_dataframe("js")
    description = description_to_dict()

    temp_dict = {}
    temp_dict['py'] = py_get_dict.get_dict(py_df)
    temp_dict['js'] = js_get_dict.get_dict(js_df)
    temp_dict['description'] = description

    return temp_dict

data = get_viz_json()
data